# Softmin Discrete Minimax Classifer for Imbalanced Datasets and Prior Probability Shifts

#### References for these algorithms:
Article [1]: *Discrete Box-Constrained Minimax Classifier for Uncertain and Imbalanced Class Proportions.* Cyprien Gilet, Susana Barbosa, Lionel Fillatre, IEEE TRANSACTIONS ON PATTERN ANALYSIS AND MACHINE INTELLIGENCE, 2020.

Article [2]: *Adjusting Decision Trees for Uncertain Class Proportions.* Cyprien Gilet, Marie Guyomard, Susana Barbosa, Lionel Fillatre, ECML/PKDD Workshop on Uncertainty in Machine Learning (WUML), 2020.

Article [3]: *Softmin Discrete Minimax Classifier for Imbalanced Classes and Prior Probability Shifts.* Cyprien Gilet, Marie Guyomard, Sébastien Destercke, Lionel Fillatre, available soon, 2023.

The authors sincerely thank Marie Guyomard for her contributions and her precious help in this project.

# Functions

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import xlrd
import random
import time
import seaborn as sn
from sklearn.cluster import KMeans
from sklearn.model_selection import KFold
from scipy.stats import bernoulli
from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from itertools import combinations 

### Class-conditional risks and global risk associated with any classifier

Compute the class-conditional risks as established in equation (5) on the paper [1]:

$$\hat{R}_k\left( \delta \right)  := \sum_{l\in\mathcal{\hat{Y}}} L_{kl}  \, \hat{\mathbb{P}}(\delta({X_{i}}) = l \mid Y_i = k).$$

In [ ]:
def compute_conditional_risk(YR, Yhat, K, L): 
    '''
    Parameters
    ----------
    YR : DataFrame
        Real labels.
    Yhat : Array
        Predicted labels.
    K : int
        Number of classes.
    L : Array
        Loss Function.

    Returns
    -------
    R : Array of floats
        Conditional risks.
    confmat : Matrix
        Confusion matrix.
    '''
   
    confmat = np.zeros((K, K))
    R = np.zeros((1, K))
    YR_liste= YR.values.tolist()
    for k in range(0, K):
        mk = YR_liste.count([k+1])
        if mk > 0:
            Ik = np.where(YR == (k + 1))
            for l in range(0, K):
                confmat[k,l] = sum(Yhat[Ik[0]]==l+1)/mk
        R[0,k] = L[k, :].dot(confmat[k, :]) 
    
    return R, confmat

Compute the global risk as established in equation (4) on the paper [1]:

$$\hat{r}\left(\delta\right)  =  \sum_{k\in\mathcal{Y}} \hat{\pi}_k  \hat{R}_k\left( \delta \right).$$

In [11]:
def compute_global_risk(R, pi):
    '''
    Parameters
    ----------
    R : Array of floats
        Conditional risks.
    pi : Array of floats
        Class proportions.

    Returns
    -------
    r : float
        Global risk.
    '''
    
    r = (pi[0].dot(R[0]))
    
    return r

### Class proportions associated with the training set

Compute the class proportions as established in equation (1) on the paper [1]:

$$\hat{\pi}_k = \frac{1}{m} \sum_{i\in\mathcal{I}} \mathbb{1}_{\{Y_i = k\}},\;\forall  k \in \mathcal{Y}.$$

In [14]:
def compute_pi(K, Y):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    Y : DataFrame
        Real labels.

    Returns
    -------
    pi : Array of floats
        Class proportions.
    '''
    
    pi = np.zeros((1,K))
    Y = Y.values.tolist()
    
    for k in range(0, K):
        pi[0,k] = Y.count([k+1])/np.shape(Y)[0]
   
    return pi

## Fit Discrete Box-Constrained Minimax Classifier

In [7]:
def fit_DMC(XTrain, YRTrain, K, L, discretization, param_kmeans, param_DT, N, eps, Box):
    '''
    Parameters
    ----------
    XTrain : DataFrame
        Features of the training samples.
    YRTrain : DataFrame
        Real labels of the trainind samples.
    K : int
        Number of classes.
    L : Array
        Loss function.
    discretization : str
        Type of discrimination : {'kmeans', 'DT'}.
    param_kmeans : list of parameters for the k-means discretization.
            {None, [nbFoldsfitT, nbT, optionPlot, T_max]}
                nbFoldsfitT : int
                    Number of iterations in the cross-validation procedure for determining the number of centroids.
                nbT : int
                    Number of centroids to be tested .
                optionPlot : int {0,1}
                    1 plots figure,   0: does not plot figure.
                T_max : int
                    Maximum number of centroids.
    param_DT : List of parameters for Decision Trees discretization
            {None, [depth, nbFolds_FB_DT, optionPlot]}
                depth : List
                    List of maximal depth to be tested.
                nbFolds_FB_DT : int
                    Number of folds for the Cross Validation of the force_brute_DT function.
                optionPlot : int {0,1}
                    1 plots figure,   0: does not plot figure.
    N : int
        Number of iterations for the projected subgradient algorithm.
    eps : float
        Maximum gap risk between training and validation sets.
    T : int
        Optimal number of centroids for the discretization process.
    Box : Array
       {'none', matrix} : Box-constraint on the priors.

    Returns
    -------
    piStarDMC : Array of floats
        Least favorable priors.
    pHatDMC : Array of floats
        Probability estimate of observing the features profile.
    discretization_model : Model Object
        Discretization process (kmeans or Decision Tree Model)
    V_iter : Array of floats
        Value of the V function at each iteration in the projected subgradient algorithm.
    T_opt_DMC : int
        Number of discrete profiles.

    '''

    print("Fit DMC ...")
    
    # DISCRETIZATION OF THE FEATURES
    
    if discretization=="kmeans" :
        
        nbFoldsfitT = param_kmeans[0]
        nbT = param_kmeans[1]
        optionPlot = param_kmeans[2]
        T_max = param_kmeans[3]
    
        T_opt_DMC = fit_NBcentroids_T(XTrain, YRTrain, K, L, nbFoldsfitT, nbT, eps, optionPlot, T_max)
        kmeansDMC = KMeans(n_clusters = int(T_opt_DMC))
        kmeansDMC.fit(XTrain)
        discretization_model = kmeansDMC
        XDTrain = kmeansDMC.labels_ + 1

    if discretization=="DT" :
        
        depth = param_DT[0]
        nbFolds_FB_DT = param_DT[1]
        optionPlot = param_DT[2]

        # Fit the depth of the decision tree with respect to eps
        depth_opt = DT_force_brute(XTrain, YRTrain, K, L, nbFolds_FB_DT, depth, eps)
        
        # Fit the decision tree with depth = depth_opt
        modele_Discr = DecisionTreeClassifier(ccp_alpha=0.0, 
                                           class_weight=None, 
                                           criterion='gini', 
                                           max_depth= depth_opt, 
                                           splitter='best').fit(XTrain, YRTrain)
        discretization_model = modele_Discr
        
        # Discretization of the features using the fitted decision tree
        XDTrain = discretisation_DT(XTrain, modele_Discr)
        T_opt_DMC = np.max(XDTrain)
   

    # PROJECTED SUBGRADIENT ALGORITHM FOR COMPUTING piStarDMC:
    pHatDMC = compute_pHat(XDTrain, YRTrain, K, T_opt_DMC)
    piStarDMC, rStarDMC, RStarDMC, V_iter, stockpi = compute_piStar(pHatDMC, YRTrain, K, L, T_opt_DMC, N, optionPlot, Box)

    ### Uncomment for Allowing to smooth the values of piStar at the last iterations and to avoid some coordinates to
    ### vanish when dealing with a large number of classes:
    # piStarMean = piStarDMC.copy()
    # for k in range(K):
    #     piStarMean[0,k] = np.mean(stockpi[k,N-20:N])
    # piStarDMC = piStarMean/np.sum(piStarMean)
    
    return piStarDMC, pHatDMC, discretization_model, V_iter, T_opt_DMC


### Functions used in fit_DMC

##### Fit the number of centroids when diescretizing the features using the Kmeans algorithm

The function *fit_NBcentroids_T* allows to compute the optimal number of centroids for achieving an acceptable generalization error. To this aim, this function performs a $nbFolds$-cross-validation procedure on the training set to select the optimal number of centroids (see Flowchart in Fig.4 in the paper [1]). This function calls the function *EvalCentroidsT_fold_f* at each iteration of the cross-validation procedure.

In [9]:
def fit_NBcentroids_T(X, YR, K, L, nbFolds, nbT, eps, optionPlot, T_max):
    '''
    Parameters
    ----------
    X : DataFrame
        Features.
    YR : DataFrame
        Real labels.
    K : int
        Number of classes.
    L : Array
        Loss function.
    nbFolds : int
        Number of iterations in the cross-validation procedure on the traing set.
    nbT : int
        Number of centroids to be tested .
    eps : float
        Maximum gap risk between training and validation sets.
    optionPlot : int {0,1}
        1: plot figure,   0: does not plot figure.
    T_max : int
        Largest number of centroids to be tested (T_max < number of training instances)

    Returns
    -------
    T_opt : int
        Optimal number of centroids.

    '''
    
    print('-> Fit the number of centroids:')
    kf = KFold(n_splits=nbFolds)
    stockrTrain = np.zeros((nbFolds,nbT))
    stockrTest = np.zeros((nbFolds,nbT))
    stockT = np.zeros((nbFolds,nbT))
    
    kf = KFold(n_splits=nbFolds)
    
    checkProp = 0
    while checkProp < nbFolds:
        checkProp = 0
        kf = KFold(n_splits=nbFolds, shuffle=True, random_state=True)
        for train, test in kf.split(X):
            XTrain, XTest, YRTrain, YRTest = X.loc[train,:], X.loc[test,:], YR.loc[train,:], YR.loc[test,:]
            Xtrain = XTrain.reset_index()
            Xtrain.drop(['index'], axis='columns', inplace=True)
            Xtest = XTest.reset_index()
            Xtest.drop(['index'], axis='columns', inplace=True)
            YRtrain = YRTrain.reset_index()
            YRtrain.drop(['index'], axis='columns', inplace=True)
            YRtest = YRTest.reset_index()
            YRtest.drop(['index'], axis='columns', inplace=True)
            # Compute the proportions
            piTest = compute_pi(K, YRtest)
            if piTest[0,1] >= 0.00006:
                checkProp = checkProp + 1
    
    f = 0
    for train, test in kf.split(X):
        print('---> Sub-cross-validation procedure: Processing fold f =', f)
        XTrain, XTest, YRTrain, YRTest = X.loc[train,:], X.loc[test,:], YR.loc[train,:], YR.loc[test,:]
        stockrTrain[f,:], stockrTest[f,:], stockT[f,:] = EvalCentroidsT_fold_f(XTrain,YRTrain,XTest,YRTest,K,L,nbT, T_max)
        f = f + 1
    
    # select T_opt
    rTrainAv = np.mean(stockrTrain, axis = 0)
    rTestAv = np.mean(stockrTest, axis = 0)
    stockTAv = np.mean(stockT, axis = 0)
    
    indT = np.where(np.abs(rTrainAv - rTestAv) <= eps)
    if np.shape(indT)[1] > 0:
        T_opt = stockTAv[np.argmin(rTrainAv[indT])]
    else:
        T_opt = K
    
    # plot figure risk for all T
    if optionPlot == 1:
        rTrainSTD = np.std(stockrTrain, axis = 0)
        rTestSTD = np.std(stockrTest, axis = 0)

        plt.plot(stockTAv, rTrainAv, color = 'blue', label='Training subsets')
        plt.plot(stockTAv, (rTrainAv + rTrainSTD), color = 'blue', linestyle = "--")
        plt.plot(stockTAv, (rTrainAv - rTrainSTD), color = 'blue', linestyle = "--")

        plt.plot(stockTAv, rTestAv, color = 'orange', label='Validation sets')
        plt.plot(stockTAv, (rTestAv + rTestSTD), color = 'orange', linestyle = "--")
        plt.plot(stockTAv, (rTestAv - rTestSTD), color = 'orange', linestyle = "--")
        plt.axvline(x = T_opt, linewidth=2, color='g')

        font = {'weight':'normal','size':16}
        plt.title('Computation of T_opt', fontdict=font)
        plt.xlabel('Number of centroids $T$', fontdict=font)
        plt.ylabel('Global risk $r$', fontdict=font)
        
        plt.grid(True)
        plt.grid(which='minor', axis='x', ls='-.')
        plt.legend(loc=2, shadow=True)

        plt.show()
        
    print('-> Optimal number of centroids T_opt (with respect to epsT) =', T_opt)
    
    return int(T_opt)

The following function EvalCentroidsT_fold_f compute the global risk of error on both the training on validation set for $nbT$ different number of centroids (see Flowchart in Fig.4 in the paper [1]).

In [ ]:
def EvalCentroidsT_fold_f(XTrain, YRTrain, XTest, YRTest, K, L, nbT, T_max):
    '''
    Parameters
    ----------
    XTrain : DataFrame
        Features of the training sample.
    YRTrain : DataFrame
        Real labels of the training sample.
    XTest : DataFrame
        Features of the validation sample.
    YRTest : DataFrame
        Real labels of the validation sample.
    K : int
        Number of classes.
    L : Matrix
        Loss Function.
    nbT : int
        Number of centroids to be tested.
    T_max : int
        Number maximal of centroids.

    Returns
    -------
    rTrain : Array of floats
        Global risks for the training sample for each number of centroids.
    rTest : Array of floats
        Global risk for the validation sample for each number of centroids.
    stockT : Vector
        Number of centroids that have been tested.

    '''

    m = np.shape(XTrain)[0]
    
    # Establish all the numbers of centroids T to be tested
    if m > T_max:
        Tmax = T_max
    else:
        Tmax = m
    a = 1/(nbT-1) * math.log(Tmax/K)
    b = math.log(K) - a
    stockT = np.zeros((1,nbT))
    for T in range(0, nbT):
        stockT[0,T] = np.round(math.exp(a*(T+1) + b))
    stockT[0,0] = K
    stockT[0,nbT-1] = Tmax
    
    rTrain = np.zeros((1,nbT))
    rTest = np.zeros((1,nbT))
    interT = 0
    
    for TT in stockT[0]:    
        T = int(TT)
        print('------> Processing number of centroids T =', T)
        kmeans = KMeans(n_clusters = T)
        kmeans.fit(XTrain)
        XDTrain = kmeans.labels_ + 1
        XDTest = kmeans.predict(XTest) + 1
        
        piTrain = compute_pi(K,YRTrain)
        pHat = compute_pHat(XDTrain, YRTrain, K, T)
        YhatTrain = delta_Bayes_discret(XDTrain,pHat,piTrain,K,L)
        RTrain, confmat = compute_conditional_risk(YRTrain, YhatTrain, K, L)
        rTrain[0,interT] = compute_global_risk(RTrain, piTrain)
        
        YhatTest = delta_Bayes_discret(XDTest,pHat,piTrain,K,L)
        RTest, confmat = compute_conditional_risk(YRTest, YhatTest, K, L)
        piTest = compute_pi(K,YRTest)
        rTest[0,interT] = compute_global_risk(RTest, piTest)
        
        interT = interT + 1
        
    return rTrain, rTest, stockT

##### Functions used if the discretization of the features is processed using decision trees
The following functions refer to our paper [2].

The function *DT_force_brute* allows to compute the optimal depth of the decision tree for achieving an acceptable generalization error. To this aim, this function performs a $nbFolds$-cross-validation procedure on the training set to select the optimal depth. 

In [ ]:
def DT_force_brute(X, YR, K, L, nbFolds, depth, eps):
    '''
    Parameters
    ----------
    X : DataFrame
        Features.
    YR : DataFrame
        Real labels.
    K : int
        Number of classes.
    L : Matrix
        Loss function.
    nbFolds : int
        Number of iteration for the cross-validation on the training set.
    depth : Array
        List of depths to be tested.
    eps : float
        Maximum gap risk between training and validation sets.
        

    Returns
    -------
    depth_opt : int
        Optimal maximal depth to use.

    '''
    
    kf = KFold(n_splits=nbFolds)
    print("-> Search of the optimal depth:")

    stockrTrain = np.zeros((nbFolds,len(depth)))
    stockrTest = np.zeros((nbFolds,len(depth)))

    
    f=0  
    # For all folds of CV
    for train, test in kf.split(X):
        print('---> Sub-cross-validation procedure: Processing fold f =', f)
        #print("processing fold f =", f) 
        XTrain, XTest, YRTrain, YRTest = X.loc[train,:], X.loc[test,:], YR.loc[train,:], YR.loc[test,:]
        Xtrain = XTrain.reset_index()
        Xtrain.drop(['index'], axis='columns', inplace=True)
        Xtest = XTest.reset_index()
        Xtest.drop(['index'], axis='columns', inplace=True)
        YRtrain = YRTrain.reset_index()
        YRtrain.drop(['index'], axis='columns', inplace=True)
        YRtest = YRTest.reset_index()
        YRtest.drop(['index'], axis='columns', inplace=True)

        piTrain = compute_pi(K, YRtrain)
        piTest = compute_pi(K, YRtest)

        i=0
        # for all possible depth
        for d in depth:
            print('------> Processing depth =', d)
            # Fit the model
            model_DT = Decision_Tree_fit(Xtrain, YRtrain, d, None)
            # Test on the training sample
            YhatTrainDT = Decision_Tree_predict(Xtrain,model_DT)
            RDT_Train, confmatTrainDT = compute_conditional_risk(YRtrain, YhatTrainDT, K, L)
            stockrTrain[f,i] = compute_global_risk(RDT_Train, piTrain)
            # Test on the validation sample
            YhatTestDT = Decision_Tree_predict(Xtest,model_DT)
            RDT_Test, confR_DT_Test = compute_conditional_risk(YRtest, YhatTestDT, K, L)
            stockrTest[f,i] = compute_global_risk(RDT_Test, piTest)

            i=i+1
        
        f=f+1
            

        
    
    # select depth_opt
    rTrainAv = np.mean(stockrTrain, axis = 0)
    rTestAv = np.mean(stockrTest, axis = 0)
    
    indT = np.where(np.abs(rTrainAv - rTestAv) <= eps)
    if np.shape(indT)[1] > 0:
        depth_opt = depth[np.argmin(rTrainAv[indT])]
    else:
        depth_opt = depth[0]
        print('-> Warning: there is no depth achieving abs(rTrain - rTest) < eps')
    
        
        
    # plot figure risk for all T
    if optionPlot == 1:
        rTrainSTD = np.std(stockrTrain, axis = 0)
        rTestSTD = np.std(stockrTest, axis = 0)

        plt.plot(depth, rTrainAv, color = 'blue', label='Training subsets')
        plt.plot(depth, (rTrainAv + rTrainSTD), color = 'blue', linestyle = "--")
        plt.plot(depth, (rTrainAv - rTrainSTD), color = 'blue', linestyle = "--")

        plt.plot(depth, rTestAv, color = 'orange', label='Validation sets')
        plt.plot(depth, (rTestAv + rTestSTD), color = 'orange', linestyle = "--")
        plt.plot(depth, (rTestAv - rTestSTD), color = 'orange', linestyle = "--")
        plt.axvline(x = depth_opt, linewidth=2, color='g')

        font = {'weight':'normal','size':16}
        plt.title('Computation of the optimal depth', fontdict=font)
        plt.xlabel('Depth of the decision tree', fontdict=font)
        plt.ylabel('Global risk $r$', fontdict=font)
        
        plt.grid(True)
        plt.grid(which='minor', axis='x', ls='-.')
        plt.legend(loc=2, shadow=True)

        plt.show()
    
    print('-> Optimal Depth (with respect to epsT) =', depth_opt)

    return depth_opt

Fit the decision tree for a given depth (function called at each iteration of the cross-validation procedure in the function DT_force_brute).

In [ ]:
def Decision_Tree_fit(XTrain, YRTrain, depth, classweight):
    '''
    Parameters
    ----------
    XTrain : DataFrame
        Features of the training sample.
    YRTrain : DataFrame
        Real labels of the training sample.
    depth : int
        Depth of the decision tree.
    classweight : str
        Weights associated with classes
        The “balanced” mode uses the values of y to automatically adjust 
        weights inversely proportional to class frequencies in the input data as 
        n_samples / (n_classes * np.bincount(y)).

    Returns
    -------
    model_DT : Decision Tree Classifier Model
        The Decision Tree model fitted.

    '''
    
    model_DT = DecisionTreeClassifier(criterion = 'gini', 
                                      splitter = 'best', 
                                      max_depth = depth, 
                                      class_weight = classweight).fit(XTrain, YRTrain)
    return model_DT

Prediction using the decision tree (function used for evaluating the generalization error of the decision tree in the function DT_force_brute).

In [ ]:
def Decision_Tree_predict(XTest, model):
    '''
    Parameters
    ----------
    XTest : DataFrame
        Features of the testing sample.
    model : Decision Tree Classifier Model
        The Decision Tree model used for the prediction.

    Returns
    -------
    Yhat_DT : Vector
        Labels estimated by the Decision Tree Classifier.

    '''
    
    Yhat_DT = model.predict(XTest)
    
    return Yhat_DT

Discretization of the features using the decision tree.

In [6]:
def discretisation_DT(X, modele) :
    '''
    Parameters
    ----------
    X : DataFrame
        Features.
    modele : Decision Tree Classifier Model
        Decidion Tree model.

    Returns
    -------
    Xdiscr : Vector
        Discretised features.

    '''
    Xdiscr = DecisionTreeClassifier.apply(modele, X, check_input=True)
    # Rename each leaf
    l=1
    for i in np.unique(Xdiscr):
        for j in range(len(Xdiscr)) :
            if Xdiscr[j]==i :
                Xdiscr[j]=l
        l+=1
    return Xdiscr

##### Function for computing pHat

pHat refers to the equation (6) in the paper [1]:

$$\hat{p}_{kt}:=\frac{1}{m_k}\sum_{i \in \mathcal{I}_k}\mathbb{1}_\left\{X_i=x_t\right\}.$$

In [1]:
def compute_pHat(XDTrain, YRTrain, K, T):  
    '''
    Parameters
    ----------
    XDTrain : Vector
        Features of the training set after discretization.
    YRTrain : DataFrame
        Real labels of the training set.
    K : int
        Number of classes.
    T : int
        Number of discrete profiles.

    Returns
    -------
    pHat : Array of floats
        Probability estimate of observing the features profile in each class.

    '''

    pHat = np.zeros((K, T))
    YRTrain_list = YRTrain.values.tolist()
    
    for k in range(0, K):
        Ik = np.where(YRTrain == (k + 1))[0]
        mk = YRTrain_list.count([k+1])
        
        for t in range(0, T):
            pHat[k,t] = sum(XDTrain[Ik]==t+1)/mk
    
    return pHat

##### Projected subgradient algorithm 

The following function allows to compute the least favorable priors $\pi^{\star}$ using the procedure presented in equation (24) and in Algorithm 1 in the paper [1]:

$$\pi^{(n+1)} = \mathrm{P}_{\mathbb{U}}\left( \pi^{(n)} + \frac{\gamma_n}{\eta_{n}} \, g^{(n)}  \right).$$

In [8]:
def compute_piStar(pHat, YRTrain, K, L, T, N, optionPlot, Box):
    '''
    Parameters
    ----------
    pHat : Array of floats
        Probability estimate of observing the features profile in each class.
    YRTrain : Dataframe
        Real labels of the training set.
    K : int
        Number of classes.
    L : Array
        Loss Function.
    T : int
        Number of discrete profiles.
    N : int
        Number of iterations in the projected subgradient algorithm.
    optionPlot : int {0,1}
        1 plots figure,   0: does not plot figure.
    Box : Array
        {'none', matrix} : Box-constraints on the priors.

    Returns
    -------
    piStar : Array of floats
        Least favorable priors.
    rStar : float
        Global risks.
    RStar : Array of float
        Conditional risks.
    V_iter : Array
        Values of the V function at each iteration.
    stockpi : Array
        Values of pi at each iteration.

    '''
    
    print('-> Compute the least favorable priors piStar')
    
    # IF BOX-CONSTRAINT == NONE (PROJECTION ONTO THE SIMPLEX)
    if isinstance(Box, str) == True :
        pi = compute_pi(K,YRTrain)
        rStar = 0
        piStar = pi
        RStar = 0
        
        V_iter = []
        stockpi = np.zeros((K, N))
        
        for n in range(1, N+1):
            # Compute subgradient R at point pi (see equation (21) in the paper)
            lambd = np.dot(L, pi.T * pHat)
            R = np.zeros((1, K))
            for k in range(0, K):
                mu_k = 0
                for t in range(0, T):
                    lbar = np.argmin(lambd[:,t])
                    mu_k = mu_k + L[k,lbar] * pHat[k,t]
                R[0,k] = mu_k
                stockpi[k,n-1] = pi[0,k]
            r = compute_global_risk(R, pi) 
            V_iter.append(r)
            if r > rStar:
                rStar = r
                piStar = pi
                RStar = R 
            # Update pi for iteration n+1
            gamma = 1/n
            eta = np.maximum(float(1),np.linalg.norm(R))
            w = pi + (gamma/eta)*R
            pi = proj_simplex_Condat(K,w)
            
            
        # Check if pi_N == piStar
        lambd = np.dot(L, pi.T * pHat)
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:,t])
                mu_k = mu_k + L[k,lbar] * pHat[k,t]
            R[0,k] = mu_k
            stockpi[k,n-1] = pi[0,k]
        r = compute_global_risk(R, pi)
        if r > rStar:
            rStar = r
            piStar = pi
            RStar = R
            
        if optionPlot == 1:
            graph_convergence(V_iter)
        
        
    # IF BOX-CONSTRAINT
    if isinstance(Box, str) == False :
        pi = compute_pi(K,YRTrain)
        rStar = 0
        piStar = pi
        RStar = 0
        
        V_iter = []
        stockpi = np.zeros((K, N))
        
        for n in range(1, N+1):
            # Compute subgradient R at point pi (see equation (21) in the paper)
            lambd = np.dot(L, pi.T * pHat)
            R = np.zeros((1, K))
            for k in range(0, K):
                mu_k = 0
                for t in range(0, T):
                    lbar = np.argmin(lambd[:,t])
                    mu_k = mu_k + L[k,lbar] * pHat[k,t]
                R[0,k] = mu_k
                stockpi[k,n-1] = pi[0,k]
            r = compute_global_risk(R, pi) 
            V_iter.append(r)
            if r > rStar:
                rStar = r
                piStar = pi
                RStar = R 
            # Update pi for iteration n+1
            gamma = 1/n
            eta = np.maximum(float(1),np.linalg.norm(R))
            w = pi + (gamma/eta)*R
            pi = proj_onto_U(w, Box, K)
            
            
        # Check if pi_N == piStar
        lambd = np.dot(L, pi.T * pHat)
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:,t])
                mu_k = mu_k + L[k,lbar] * pHat[k,t]
            R[0,k] = mu_k
            stockpi[k,n-1] = pi[0,k]
        r = compute_global_risk(R, pi)
        if r > rStar:
            rStar = r
            piStar = pi
            RStar = R
            
        if optionPlot == 1:
            graph_convergence(V_iter)
              
        
    return piStar, rStar, RStar, V_iter, stockpi

Projection onto the simplex:

This function is inspired from the article:
L.Condat, "Fast projection onto the simplex and the $\ell_1$ ball", *Mathematical Programming*, vol.158, no.1, pp. 575-585, 2016.

In [ ]:
def proj_simplex_Condat(K, pi):  
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    pi : Array of floats
        Vector to project onto the simplex.

    Returns
    -------
    piProj : List of floats
        Priors projected onto the simplex.

    '''
   
    linK = np.linspace(1, K, K)
    piProj = np.maximum(pi - np.max(((np.cumsum(np.sort(pi)[::-1]) - 1) / (linK[:]))),0)
    piProj = piProj / np.sum(piProj)
    return piProj

Projection onto the box-constrained simplex:

The function proj_onto_polyhedral_set is inspired from the article:
K. E. Rutkowski, “Closed-form  expressions for projectors ontopolyhedral sets in hilbert spaces, ”SIAM Journal on Optimization, vol. 27, pp. 1758–1771, 2017.

In [ ]:
def proj_onto_U(pi, Box, K) :
    '''
    Parameters
    ----------
    pi : Array of floats
        Vector to project onto the box-constrained simplex..
    Box : Matrix
        {'none', matrix} : Box-constraint on the priors.
    K : int
        Number of classes.

    Returns
    -------
    pi_new : Array of floats
            Priors projected onto the box-constrained simplex.

    '''
    
    check_U = 0
    if pi.sum() ==1 :
        for k in range(K) :
            if (pi[0][k] >= Box[k,0]) & (pi[0][k] <= Box[k,1]) :
                check_U = check_U + 1
    
    if check_U == K :
        pi_new = pi

      
    if check_U < K :
        pi_new = proj_onto_polyhedral_set(pi, Box, K)
    
    return pi_new





def num2cell(a):
    if type(a) is np.ndarray:
        return [num2cell(x) for x in a]
    else:
        return a 

    
    
    

def proj_onto_polyhedral_set(pi, Box, K) :
    '''
    Parameters
    ----------
    pi : Array of floats
        Vector to project onto the box-constrained simplex.
    Box : Array
        {'none', matrix} : Box-constraint on the priors.
    K : int
        Number of classes.

    Returns
    -------
    piStar : Array of floats
            Priors projected onto the box-constrained simplex.

    '''
    
    # Verification of constraints
    for i in range(K) :
        for j in range(2) :
            if Box[i,j] < 0 :
                Box[i,j] = 0
            if Box[i,j] > 1 :
                Box[i,j] = 1

    # Generate matrix G:
    U = np.concatenate((np.eye(K), -np.eye(K), np.ones((1,K)), -np.ones((1,K))))            
    eta = Box[:,1].tolist() + (-Box[:,0]).tolist() + [1] + [-1]

    n = U.shape[0]
    
    G = np.zeros((n,n))
    for i in range(n) :
        for j in range(n) :
            G[i,j] = np.vdot(U[i,:],U[j,:])
    
    
    # Generate subsets of {1,...,n}:
    M = (2**n)-1
    I = num2cell(np.zeros((1,M)))
    
    i = 0
    for l in range(n) :
        T = list(combinations(list(range(n)), l+1))
        for p in range(i,i+len(T)) :
            I[0][p] = T[p-i]
        i = i+len(T)
            
        
    # Algorithm    
        
    for m in range(M) :
        Im = I[0][m]
 
        Gmm = np.zeros((len(Im), len(Im)))
        ligne = 0
        for i in Im :
            colonne = 0
            for j in Im :
                Gmm[ligne,colonne] = G[i,j]
                colonne += 1
            ligne +=1
        

        if np.linalg.det(Gmm)!=0 :
            
            nu = np.zeros((2*K+2,1))
            w = np.zeros((len(Im),1))
            for i in range(len(Im)) :
                w[i] = np.vdot(pi,U[Im[i],:]) - eta[Im[i]]
            
            S = np.linalg.solve(Gmm,w) 
            
            for e in range(len(S)) :
                nu[Im[e]] = S[e]
            
            
            if np.any(nu<-10**(-10)) == False  :
                A = G.dot(nu)
                z = np.zeros((1,2*K+2))
                for j in range(2*K+2) :
                    z[0][j] = np.vdot(pi,U[j,:]) - eta[j] - A[j]
                    
                    
                if np.all(z<=10**(-10)) == True :
                    pi_new = pi
                    for i in range(2*K+2) :
                        pi_new = pi_new - nu[i]*U[i,:]

    piStar = pi_new

    # Remove noisy small calculus errors:
    piStar = piStar/piStar.sum()
    
    return piStar

Plot convergence of the projected subgradient algorithm 

In [ ]:
def graph_convergence(V_iter):
    '''
    Parameters
    ----------
    V_iter : List
        List of value of V at each iteration n.

    Returns
    -------
    Plot
        Plot of V_pibar.

    '''
    
    figConv = plt.figure(figsize=(8,4))
    plt_conv = figConv.add_subplot(1,1,1)
    V = V_iter.copy()
    V.insert(0,np.min(V))
    font = {'weight':'normal','size':16}
    plt_conv.plot(V, label='V(pi^(n))')
    plt_conv.set_xscale('log')
    plt_conv.set_ylim(np.min(V), np.max(V)+0.01)
    plt_conv.set_xlim(10**0)
    plt_conv.set_xlabel('Interation n', fontdict=font)
    plt_conv.set_title('Maximization of V over U', fontdict=font)
    plt_conv.grid(True)
    plt_conv.grid(which='minor', axis='x', ls='-.')
    plt_conv.legend(loc=2, shadow=True)

## Prediction using Discrete Box-Constrained Minimax Classifier

##### Function for computing delta_Bayes_discret

The function delta_Bayes_discret refers to the Bayes Classifier associated with any priors $\pi\in\mathbb{S}$ as detailed in the equation (19) in the paper [1]:

$${\delta}_{\pi}^B : X_{i} \mapsto \underset{l\in\mathcal{\hat{Y}}}{\mathrm{arg\,min}} \, \sum_{t\in\mathcal{T}}\sum_{k\in\mathcal{Y}} L_{kl} \, \pi_k   \, \hat{p}_{kt} \,\mathbb{1}_{\{X_{i}=x_{t} \}}.$$

In [2]:
def delta_Bayes_discret(XDTrain, pHat, pi, K, L): 
    '''
    Parameters
    ----------
    XDTrain : Array
        Features of the training sample after discretization.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array of floats
        Real class proportions.
    K : int
        Number of classes.
    L : Array
        Loss function.
    
    Returns
    -------
    Yhat : Vector
        Predicted labels.

    '''
    
    Yhat = np.zeros((np.shape(XDTrain)[0], 1))
    for i in range(0,np.shape(XDTrain)[0]):
        t = int(XDTrain[i])
        lambd = np.zeros((1,K))
        for l in range(0, K):
            for k in range(0, K):
                lambd[0,l] = lambd[0,l] + L[k,l] * pi[0,k] * pHat[k,t-1]
        lbar = np.argmin(lambd[0,:]) 
        Yhat[i,0] = lbar + 1
    return Yhat

##### Function predict_DMC

The function *predict_DMC* aims to predict the class labels of the test instances using the Discrete Minimax Classifier associated with the priors $\pi^{\star}$. This function includes the discretization process of the numeric features.

In [3]:
def predict_DMC(XTest, K, L, discretization, modele, pHatDMC, piStarDMC):
    '''
    Parameters
    ----------
    XTest : DataFrame
        Features of the testing sample.
    K : int
        Number of classes.
    L : Array
        Loss function.
    discretization : str
        The type of discretization used
    modele : kmeans or Decision Tree classifier model
        The model for the discretization
    pHatDMC : Array of floats
        Probability estimate of observing the features profile.
    piStarDMC : Array of floats
        Least favorable priors.
    
    Returns
    -------
    YhatTest : Array
        Predicted labels.

    '''
    
    if discretization == 'kmeans':
        XDTest = modele.predict(XTest) + 1
        
    if discretization == 'DT' :
         XDTest = discretisation_DT(XTest, modele)

    YhatTest = delta_Bayes_discret(XDTest, pHatDMC, piStarDMC, K, L)
    
    return YhatTest

## Plot of the function V when dealing with K=2 classes

In [ ]:
def plot2D_V_piTrain_piBar(pHat, K, L, T, piTrain, piBar):
    '''
    param:pHat: probability estimate of observing the features profiles x_t in each class k
    param:K: number of classes
    param:L: loss function
    param:T: optimal number of centroids for the discretization process
    param:piTrain: class proportions on the training set
    param:piBar: least favorable priors
    return:rpiTrain: global risk on the training set
    return:rpiBar: V(piBar)     
    '''
    
    xx = np.sort(np.insert(np.linspace(0,1,100), [2, 2], [piTrain[0,0], piBar[0,0]]))
    stock_pi = np.zeros((np.shape(xx)[0], K))
    stock_r = np.zeros((1, np.shape(xx)[0]))
    
    i = 0
    for x in xx:
        stock_pi[i,0] = x
        stock_pi[i,1] = 1 - x
        i = i + 1
    
    for i in range(0, np.shape(xx)[0]):
        lambd = np.zeros((K,T))
        for l in range(0, K):
            for t in range(0, T):
                for k in range(0, K):
                    lambd[l,t] = lambd[l,t] + L[k,l] * stock_pi[i,k] * pHat[k,t]
        R = np.zeros((1, K))
        for k in range(0, K):
            mu_k = 0
            for t in range(0, T):
                lbar = np.argmin(lambd[:,t])
                mu_k = mu_k + L[k,lbar] * pHat[k,t]
            R[0,k] = mu_k
        stock_r[0,i] = np.dot(R[0], stock_pi[i,:])
        
        if stock_pi[i,0] == piTrain[0,0]:
            RpiTrain = R
            rpiTrain = np.dot(piTrain[0], RpiTrain[0])
            stock_r_piTrain = np.dot(stock_pi, (RpiTrain).T)
        
        if stock_pi[i,0] == piBar[0,0]:
            RpiBar = R
            rpiBar = np.dot(piBar[0],RpiBar[0])
            stock_r_piBar = np.dot(stock_pi, (RpiBar).T)
    
    # Figure
    
    plt.plot(xx, stock_r[0], color = 'blue', label='Function V')
    plt.plot(xx, stock_r_piTrain, color = 'orange', label='Discrete Bayes Classifier')
    plt.vlines(x = piTrain[0,0], ymin=0, ymax=rpiTrain, linewidth=2, color='orange', linestyle = "--")
    plt.plot(xx, stock_r_piBar, color = 'green', label='Discrete Minimax Classifier')
    plt.vlines(x = piBar[0,0], ymin = 0, ymax = rpiBar, linewidth = 2, color = 'green', linestyle = "--")
    
    plt.rc('text', usetex=True)
    font = {'weight':'normal','size':20}
    #plt.title('Discrete empirical Bayes risk as a function of the priors', fontdict=font)
    plt.xlabel('Prior $\pi_1$', fontdict=font)
    plt.ylabel('Global risk $r$', fontdict=font)
    
    plt.text(1,RpiTrain[0,0],r'$\hat{R}_1\left(\delta_{\hat{\pi}}^B\right)$',{'color':'orange','fontsize':20})
    plt.text(-0.2,RpiTrain[0,1],r'$\hat{R}_2\left(\delta_{\hat{\pi}}^B\right)$',{'color':'orange','fontsize':20})
    plt.text(piTrain[0,0]-0.02,-0.12,r'$\hat{\pi}_1$',{'color':'orange','fontsize':20})
    
    plt.text(1,RpiBar[0,0],r'$\hat{R}_1\left(\delta_{\bar{\pi}}^B\right)$',{'color':'green','fontsize':20})
    plt.text(-0.2,RpiBar[0,1],r'$\hat{R}_2\left(\delta_{\bar{\pi}}^B\right)$',{'color':'green','fontsize':20})
    plt.text(piBar[0,0]-0.02,-0.12,r'$\bar{\pi}_1$',{'color':'green','fontsize':20})
    
    plt.xlim(0,1)
    plt.ylim(0)
    
    plt.grid(True)
    plt.legend(loc=1, shadow=True)
    plt.show()
    
    return rpiTrain, rpiBar

## Functions Softmin Discrete Minimax Classifier

In [ ]:
def SoftminDMC_predict(X, K, L, discretization, modele, pHat, pi, lambd):
    '''
    Parameters
    ----------
    X : DataFrame
        Features of the instances.
    K : int
        Number of classes.
    L : Array
        Loss function.
    discretization : str
        The type of discretization used
    modele : kmeans or Decision Tree classifier model
        The model for the discretization
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    
    Returns
    -------
    Yhat : Array
        Predicted labels.
    OutputProba : Array
        Estimated probabilties of each instance to belong in each class.

    '''
    
    if discretization == 'kmeans':
        XD = modele.predict(X) + 1
        
    if discretization == 'DT' :
        XD = discretisation_DT(X, modele)

    
    Yhat = np.zeros((np.shape(XD)[0], 1))
    OutputProba = np.zeros((np.shape(XD)[0], K))
    
    for i in range(0,np.shape(XD)[0]):
        t = int(XD[i])
        F = np.zeros((1,K))
        NUM_SIGM = np.zeros((1,K))
        for l in range(0, K):
            for k in range(0, K):
                F[0,l] = F[0,l] + L[k,l] * pi[0,k] * pHat[k,t-1]
            NUM_SIGM[0,l] = np.exp(-lambd * F[0,l])
        OutputProba[i,:] = NUM_SIGM/np.sum(NUM_SIGM)
        Yhat[i,0] = np.argmax(np.random.multinomial(1,OutputProba[i,:])) + 1
    
    return Yhat, OutputProba

In [ ]:
def SoftminDMC_compute_CondRisks(K, L, pHat, pi, lambd):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    L : Array
        Loss function.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array (a unique line) of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    
    Returns
    -------
    R_softminDMC : Array
        Values of the class-conditional risks at the point pi.
        
    '''
    
    T = np.shape(pHat)[1]
    F = np.zeros((K,T))
    NUM_SIGM = np.zeros((K,T))
    for t in range(0, T):
        for l in range(0, K):
            for j in range(0, K):
                F[l,t] = F[l,t] + L[j,l] * pHat[j,t] * pi[0,j]
            NUM_SIGM[l,t] = np.exp(-lambd * F[l,t])
    DENUM_SIGM = np.sum(NUM_SIGM, axis=0)

    R_softminDMC = np.zeros((1, K))
    for k in range(0, K):
        mu_k = 0
        for t in range(0, T):
            for l in range(0, K):
                sigma_l = NUM_SIGM[l,t]/DENUM_SIGM[t]
                mu_k = mu_k + L[k,l] * pHat[k,t] * sigma_l
        R_softminDMC[0,k] = mu_k

    return R_softminDMC[0]

In [ ]:
def compute_normGpi(K, L, pHat, pi, lambd):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    L : Array
        Loss function.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array (a unique line) of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    
    Returns
    -------
    normGpi : Float
        Value of the norm of G(pi).
        
    '''
    
    R_softminDMC = SoftminDMC_compute_CondRisks(K, L, pHat, pi, lambd)
    V_pi = pi[0].dot(R_softminDMC)
    normGpi = np.linalg.norm(R_softminDMC-V_pi)

    return normGpi

In [ ]:
def SoftminDMC_compute_G_root_MonteCarlo(K, L, pHat, lambd, N, Alpha, epsilon):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    L : Array
        Loss function.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array (a unique line) of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    N : int
        Maximum number of iterations for the imulated annealing algorithm.
    Alpha : Array
        Parameter for the Dirichlet distribution.
    epsilon : Float
        Positive threshold allowing to accept piSTAR.
    
    Returns
    -------
    piSTAR : Array
        The priors for which the application G vanishes.
        
    '''
    
    c = np.sum(np.sum(L,1)*np.sum(L,1))
    piSTAR = np.random.dirichlet(Alpha,1)
    normGpiStar = compute_normGpi(K, L, pHat, piSTAR, lambd)
    pi = np.copy(piSTAR)
    normGpi = normGpiStar
    
    for n in range(0, N):
        
        if normGpi < normGpiStar:
            normGpiStar = normGpi
            piSTAR[:] = pi[:]
            print('At iteration n =', n)
            print('----> normGpiStar =', normGpiStar)
            if normGpiStar <= epsilon:
                break
                
        tau = np.random.dirichlet(Alpha,1)
        normGtau = compute_normGpi(K, L, pHat, tau, lambd)
        pi[:] = tau[:]
        normGpi = normGtau
            
        
    print('Final iteration at n =', n)
    print('----> normGpiStar =', normGpiStar)
        
    return piSTAR

In [ ]:
def SoftminDMC_compute_G_root_ascent_Step(K, L, pHat, lambd, N, epsilon):
    '''
    Parameters
    ----------
    K : int
        Number of classes.
    L : Array
        Loss function.
    pHat : Array of floats
        Probability estimate of observing the features profile.
    pi : Array (a unique line) of floats
        Priors.
    lambd : Float
        Positive Temperature parameter.
    N : int
        Maximum number of iterations for the imulated annealing algorithm.
    epsilon : Float
        Positive threshold allowing to accept piSTAR.
    
    Returns
    -------
    piSTAR : Array
        The priors for which the application G vanishes.
        
    '''
    
    #np.random.seed(407)
    pi_init = np.ones((1,K))/K
    normGpiStar = compute_normGpi(K, L, pHat, pi_init, lambd)
    pi = pi_init.copy()
    piSTAR = pi_init.copy()


    for n in range(1,N):

        R_softminDMC = SoftminDMC_compute_CondRisks(K, L, pHat, pi, lambd)
        V_pi = pi[0].dot(R_softminDMC)
        Gpi = R_softminDMC - V_pi
        normGpi = np.linalg.norm(Gpi)

        if normGpi < normGpiStar:
            normGpiStar = normGpi
            piSTAR[:] = pi[:]
            if normGpiStar <= epsilon:
                break

        gamma = 1/n
        eta = np.maximum(float(1),normGpi)
        w = pi + (gamma/eta)*Gpi   
        pi = proj_simplex_Condat(K,w)
        
    
    print('Final iteration at N =', n)
    print('----> normGpiStar =', normGpiStar)

    return piSTAR